   - Date-wise Maximum Minimum
   - Dictionary to Dataframe
   - Column Values Shift
   - Identify Increasing or Decreasing Values after GroupBy

# **Date-wise Maximum Minimum**

In [ ]:
import pandas as pd
d = {'id': [1, 2, 3], 'datatime': ['16-12-19 13:23', '16-12-19 17:45', '17-12-19 13:23'], 'temp': [35,41,32]}
df = pd.DataFrame(data=d)
df = df.set_index('id')
df

,datatime,temp
id,,
1,16-12-19 13:23,35
2,16-12-19 17:45,41
3,17-12-19 13:23,32


In [ ]:
from datetime import datetime
df['NewDate'] = pd.to_datetime(df['datatime'],format='%d-%m-%y %H:%M')
df

,datatime,temp,NewDate
id,,,
1,16-12-19 13:23,35,2019-12-16 13:23:00
2,16-12-19 17:45,41,2019-12-16 17:45:00
3,17-12-19 13:23,32,2019-12-17 13:23:00


In [ ]:
df['Date Only']=df['NewDate'].dt.date
df

,datatime,temp,NewDate,Date Only
id,,,,
1,16-12-19 13:23,35,2019-12-16 13:23:00,2019-12-16
2,16-12-19 17:45,41,2019-12-16 17:45:00,2019-12-16
3,17-12-19 13:23,32,2019-12-17 13:23:00,2019-12-17


In [ ]:
import numpy as np
data = (df.set_index('Date Only').groupby(level=0)['temp']
          .agg([('Minimum',np.min),('Maximum',np.max)]))
data

,Minimum,Maximum
Date Only,,
2019-12-16,35,41
2019-12-17,32,32


In [ ]:
df.groupby('Date Only')['Date Only','temp'].min()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  """Entry point for launching an IPython kernel.


,Date Only,temp
Date Only,,
2019-12-16,2019-12-16,35
2019-12-17,2019-12-17,32


# **Dictionary to Dataframe**

In [ ]:
Data = {'zip': '94404', 'sector': 'Healthcare', 'fullTimeEmployees': 11800, 'circulatingSupply': None, 'startDate': None, 'regularMarketDayLow': 67.99, 'priceHint': 2, 'currency': 'USD'}

df = pd.DataFrame([Data])
df


,zip,sector,fullTimeEmployees,circulatingSupply,startDate,regularMarketDayLow,priceHint,currency
0,94404,Healthcare,11800,None,None,67.99,2,USD


In [ ]:
d={'year':[2019,2018,2017],'B':[10,5,17]}
df=pd.DataFrame(data=d)

print(df)

   year   B
0  2019  10
1  2018   5
2  2017  17


# **Column Values Shift**

In [ ]:
import numpy as np

df = df.convert_dtypes()
df['New'] = df.B.shift(-1)
df

,year,B,New
0,2019,10,5
1,2018,5,17
2,2017,17,<NA>


# **Identify Increasing or Decreasing Values after GroupBy**

In [ ]:
import pandas as pd
d = {'id': [1, 1, 1, 2, 3,3,3], 'value': [50,100,200, 10, 500,300,100,]}
df = pd.DataFrame(data=d)
df = df.set_index('id')
df

,value
id,
1,50
1,100
1,200
2,10
3,500
3,300
3,100


In [ ]:
df['new'] = df.groupby(['id'])['value'].transform(lambda x : \
                      np.where(x.diff()>0,'increase',
                      np.where(x.diff()<0,'decrease','--')))
df2 = df.copy()
df

,value,new
id,,
1,50,--
1,100,increase
1,200,increase
2,10,--
3,500,--
3,300,decrease
3,100,decrease


In [ ]:

df = df.groupby('id').new.agg(['last'])

print(df)

In [ ]:
increasingList = df[(df['last']=='increase')].index.values
print(increasingList)

[1]


In [ ]:
df=df2.copy()

df['new'] = df.groupby(['id'])['value'].transform(lambda x : \
                      np.where(x.diff()>0,'increase',
                      np.where(x.diff()<0,'decrease','--')))

df = df.groupby(['id'])['new'].apply(lambda x: ','.join(x)).reset_index()
df



,id,new
0,1,"--,increase,increase"
1,2,--
2,3,"--,decrease,decrease"


In [ ]:
df = df.drop(df[df['new'].str.contains("dec")].index.values)
df = df.drop(df[(df['new']=='--')].index.values)
df

,id,new
0,1,"--,increase,increase"
